In [4]:
import logging
import pickle
import numpy as np
from openmmtools.integrators import PeriodicNonequilibriumIntegrator
from simtk import unit
from simtk import openmm
import argparse
import os
import time
from simtk.openmm.app import PDBFile
import mdtraj as md
import sys

In [5]:
# Set up logger
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [6]:
# Define lambda functions
x = 'lambda'
DEFAULT_ALCHEMICAL_FUNCTIONS = {
                             'lambda_sterics_core': x,
                             'lambda_electrostatics_core': x,
                             'lambda_sterics_insert': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                             'lambda_sterics_delete': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                             'lambda_electrostatics_insert': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                             'lambda_electrostatics_delete': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                             'lambda_bonds': x,
                             'lambda_angles': x,
                             'lambda_torsions': x}

In [48]:
# Define simulation parameters
nsteps_eq = 125000 # 0.25 ns
nsteps_neq = 40000 # 80 ps
neq_splitting ='V R H O R V'
timestep = 2.0 * unit.femtosecond
platform_name = 'CUDA'

In [28]:
# # Read in htf
# outdir = "/data/chodera/zhangi/perses_benchmark/neq/3/2"
# with open(os.path.join(outdir, "2_vacuum.pickle"), 'rb') as f:
#     htf = pickle.load(f)
# system = htf.hybrid_system
# positions = htf.hybrid_positions

In [38]:
# Generate htf
from perses.app.relative_point_mutation_setup import PointMutationExecutor
import simtk.openmm.app as app
apo_delivery = PointMutationExecutor(f"../input/ala_vacuum.pdb", 
                              '1', 
                              '2', 
                              'THR',
                              phase='vacuum',
	                          barostat=None,
	                          periodic_forcefield_kwargs=None, 
                            forcefield_kwargs={'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'hydrogenMass' : 4 * unit.amus},
	                          nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff}
                             )
htf = apo_delivery.get_apo_htf()
system = htf.hybrid_system
positions = htf.hybrid_positions

DEBUG:perses.forcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[4, 4, 4, 4, 4, 4]
INFO:proposal_generator:Maximum atom matched after hydrogen exceptions: 2
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 7
INFO:geometry:Atom index proposal order is [18, 14, 19, 17, 13, 15, 16]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 25 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 43 angles in refere

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 7 new atoms
INFO:geometry:	reduced angle potential = 0.33211276434346354.
INFO:geometry:	reduced angle potential = 0.02842246397829647.
INFO:geometry:	reduced angle potential = 1.3739682075057418.
INFO:geometry:	reduced angle potential = 0.3043101749611611.
INFO:geometry:	reduced angle potential = 0.15126060969555058.
INFO:geometry:	reduced angle potential = 1.4793958086191608.
INFO:geometry:	reduced angle potential = 0.1697720900664228.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 25 bond forces in the no-nonbonded final system
INFO:geometry:	there are 43 angle forces in the no-nonbonded final system
INFO:geometry:	there are

...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 10.845356658370308
INFO:geometry:final reduced energy 27.66535807795398
INFO:geometry:sum of energies: 27.66535758116222
INFO:geometry:magnitude of difference in the energies: 4.967917615061879e-07
INFO:geometry:Final logp_proposal: 25.312110238677555


added energy components: [('CustomBondForce', 5.337593030122811e-05), ('CustomAngleForce', 1.5196358350712886), ('CustomTorsionForce', 0.005213740573906639), ('CustomBondForce', 9.320453706794812)]


INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:relative:Determined atom classes.
INFO:relative:Generating old system exceptions dict...
INFO:relative:Generating new system except

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
			HarmonicBondForce: 0.0340329243809594
			HarmonicAngleForce: 2.134739442948204
			PeriodicTorsionForce: 16.17613147765423
			NonbondedForce: -38.74516998365842
			AndersenThermostat: 0.0
conducting subsequent work with the following platform: CUDA
			CustomBondForce: 0.03397954845065818
			HarmonicBondForce: 4.80384535045375
			CustomAngleForce: 0.6151036078769154
			HarmonicAngleForce: 7.927881264009106
			CustomTorsionForce: 0.0
			PeriodicTorsionForce: 30.24955562508971
			NonbondedForce: -132.4925903240406
			CustomNonbondedForce: 4.704582417313629
			CustomBondForce: 0.0
			AndersenThermostat: 0.0
conducting subsequent work with the following platform: CUDA
			CustomBondForce: 0.03397954845065818
			HarmonicBondForce: 4.80384535045375
			CustomAngleForce: 0.6151036078769154
			HarmonicAngleForce: 7.927881264009106
			CustomTorsionForce: 0.0
			PeriodicTorsio

In [49]:
# Set up integrator
integrator = PeriodicNonequilibriumIntegrator(DEFAULT_ALCHEMICAL_FUNCTIONS, nsteps_eq, nsteps_neq, neq_splitting, timestep=timestep)



In [50]:
# integrator.setConstraintTolerance(1.0e-6) # default 1e-8

In [51]:
# integrator.getConstraintTolerance()

In [52]:
# Set up context
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = openmm.Context(system, integrator, platform)
context.setPeriodicBoxVectors(*system.getDefaultPeriodicBoxVectors())
context.setPositions(positions)

In [53]:
# Minimize
openmm.LocalEnergyMinimizer.minimize(context)



In [54]:
# Run neq
ncycles = 10
forward_works_master, reverse_works_master = list(), list()
forward_equil, forward_neq_old, forward_neq_new, reverse_equil, reverse_neq_old, reverse_neq_new = list(), list(), list(), list(), list(), list()
for cycle in range(ncycles):
    # Equilibrium (lambda = 0)
    _logger.info(f'Cycle: {cycle}, Starting to equilibrate at lambda = 0')
    for step in range(nsteps_eq):
        if step % 750 == 0:
            forward_equil = forward_equil[-10:]
            elapsed_time = (time.time() - initial_time) * unit.seconds
            _logger.info(f'Cycle: {cycle}, forward equil step: {step}, took: {elapsed_time / unit.seconds} seconds')
        initial_time = time.time()
        try:
            integrator.step(1)
        except:
            _logger.info("exception!")
            with open(os.path.join(args.dir, f"{i}_{args.phase}_forward_equil_pos.npy"), 'wb') as f: # change filenames when distributing jobs
                 np.save(f, forward_equil)
            sys.exit()
        
        pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
        if np.isnan(pos).any():
            raise Exception("NANS FOUND")
        old_pos = np.asarray(htf.old_positions(pos))
        forward_equil.append(old_pos)
        

    # Forward (0 -> 1)
    forward_works = [integrator.get_protocol_work(dimensionless=True)]
    for fwd_step in range(nsteps_neq):
        
        if fwd_step % 750 == 0:
            forward_neq_old = forward_neq_old[-10:]
            forward_neq_new = forward_neq_new[-10:]
            elapsed_time = (time.time() - initial_time) * unit.seconds
            _logger.info(f'Cycle: {cycle}, forward NEQ step: {fwd_step}, took: {elapsed_time / unit.seconds} seconds')
        initial_time = time.time()
        try:
            integrator.step(1)
        except:
            _logger.info("exception!")
            with open(os.path.join(args.dir, f"{i}_{args.phase}_forward_neq_old_pos.npy"), 'wb') as f: # change filenames when distributing jobs
                np.save(f, forward_neq_old)
            with open(os.path.join(args.dir, f"{i}_{args.phase}_forward_neq_new_pos.npy"), 'wb') as f: # change filenames when distributing jobs
                np.save(f, forward_neq_new)
            sys.exit()
        pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
        if np.isnan(pos).any():
            raise Exception("NANS FOUND")
        old_pos = np.asarray(htf.old_positions(pos))
        forward_neq_old.append(old_pos)
        new_pos = np.asarray(htf.new_positions(pos))
        forward_neq_new.append(new_pos)
        forward_works.append(integrator.get_protocol_work(dimensionless=True))
    forward_works_master.append(forward_works)


    # Equilibrium (lambda = 1)
    _logger.info(f'Cycle: {cycle}, Starting to equilibrate at lambda = 1')
    for step in range(nsteps_eq):
        if step % 750 == 0:
            reverse_equil = reverse_equil[-10:]
            elapsed_time = (time.time() - initial_time) * unit.seconds
            _logger.info(f'Cycle: {cycle}, forward equil step: {step}, took: {elapsed_time / unit.seconds} seconds')
        initial_time = time.time()
        try:
            integrator.step(1)
        except:
            _logger.info("exception!")
            with open(os.path.join(args.dir, f"{i}_{args.phase}_reverse_equil_pos.npy"), 'wb') as f: # change filenames when distributing jobs
                 np.save(f, reverse_equil)
            sys.exit()
        pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
        if np.isnan(pos).any():
            raise Exception("NANS FOUND")
        new_pos = np.asarray(htf.new_positions(pos))
        reverse_equil.append(new_pos)
        

    # Reverse work (1 -> 0)
    reverse_works = [integrator.get_protocol_work(dimensionless=True)]
    for rev_step in range(nsteps_neq):
        if rev_step % 750 == 0:
            reverse_neq_old = reverse_neq_old[-10:]
            reverse_neq_new = reverse_neq_new[-10:]
            elapsed_time = (time.time() - initial_time) * unit.seconds
            _logger.info(f'Cycle: {cycle}, forward NEQ step: {rev_step}, took: {elapsed_time / unit.seconds} seconds')
        initial_time = time.time()
        try:
            integrator.step(1)
        except:
            _logger.info("exception!")
            with open(os.path.join(args.dir, f"{i}_{args.phase}_reverse_neq_old_pos.npy"), 'wb') as f: # change filenames when distributing jobs
                np.save(f, reverse_neq_old)
            with open(os.path.join(args.dir, f"{i}_{args.phase}_reverse_neq_new_pos.npy"), 'wb') as f: # change filenames when distributing jobs
                np.save(f, reverse_neq_new)
            sys.exit()
        
        pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
        if np.isnan(pos).any():
            raise Exception("NANS FOUND")
        old_pos = np.asarray(htf.old_positions(pos))
        reverse_neq_old.append(old_pos)
        new_pos = np.asarray(htf.new_positions(pos))
        reverse_neq_new.append(new_pos)
        reverse_works.append(integrator.get_protocol_work(dimensionless=True))
    reverse_works_master.append(reverse_works)

INFO:root:Cycle: 0, Starting to equilibrate at lambda = 0
INFO:root:Cycle: 0, forward equil step: 0, took: 17.52524971961975 seconds
INFO:root:Cycle: 0, forward equil step: 750, took: 0.0018160343170166016 seconds
INFO:root:Cycle: 0, forward equil step: 1500, took: 0.0016024112701416016 seconds
INFO:root:Cycle: 0, forward equil step: 2250, took: 0.0016524791717529297 seconds
INFO:root:Cycle: 0, forward equil step: 3000, took: 0.0016374588012695312 seconds
INFO:root:Cycle: 0, forward equil step: 3750, took: 0.0016872882843017578 seconds
INFO:root:Cycle: 0, forward equil step: 4500, took: 0.0017266273498535156 seconds
INFO:root:Cycle: 0, forward equil step: 5250, took: 0.0017039775848388672 seconds
INFO:root:Cycle: 0, forward equil step: 6000, took: 0.0016446113586425781 seconds
INFO:root:Cycle: 0, forward equil step: 6750, took: 0.0016198158264160156 seconds
INFO:root:Cycle: 0, forward equil step: 7500, took: 0.0016372203826904297 seconds
INFO:root:Cycle: 0, forward equil step: 8250, to

INFO:root:Cycle: 0, forward equil step: 74250, took: 0.0014388561248779297 seconds
INFO:root:Cycle: 0, forward equil step: 75000, took: 0.0010135173797607422 seconds
INFO:root:Cycle: 0, forward equil step: 75750, took: 0.0016932487487792969 seconds
INFO:root:Cycle: 0, forward equil step: 76500, took: 0.0016217231750488281 seconds
INFO:root:Cycle: 0, forward equil step: 77250, took: 0.0016775131225585938 seconds
INFO:root:Cycle: 0, forward equil step: 78000, took: 0.0016596317291259766 seconds
INFO:root:Cycle: 0, forward equil step: 78750, took: 0.0018568038940429688 seconds
INFO:root:Cycle: 0, forward equil step: 79500, took: 0.0018074512481689453 seconds
INFO:root:Cycle: 0, forward equil step: 80250, took: 0.0015425682067871094 seconds
INFO:root:Cycle: 0, forward equil step: 81000, took: 0.0016736984252929688 seconds
INFO:root:Cycle: 0, forward equil step: 81750, took: 0.0015289783477783203 seconds
INFO:root:Cycle: 0, forward equil step: 82500, took: 0.0016481876373291016 seconds
INFO

INFO:root:Cycle: 0, forward NEQ step: 24000, took: 0.0026657581329345703 seconds
INFO:root:Cycle: 0, forward NEQ step: 24750, took: 0.0026655197143554688 seconds
INFO:root:Cycle: 0, forward NEQ step: 25500, took: 0.0026879310607910156 seconds
INFO:root:Cycle: 0, forward NEQ step: 26250, took: 0.0026350021362304688 seconds
INFO:root:Cycle: 0, forward NEQ step: 27000, took: 0.0028777122497558594 seconds
INFO:root:Cycle: 0, forward NEQ step: 27750, took: 0.0028030872344970703 seconds
INFO:root:Cycle: 0, forward NEQ step: 28500, took: 0.0018868446350097656 seconds
INFO:root:Cycle: 0, forward NEQ step: 29250, took: 0.002919435501098633 seconds
INFO:root:Cycle: 0, forward NEQ step: 30000, took: 0.0017383098602294922 seconds
INFO:root:Cycle: 0, forward NEQ step: 30750, took: 0.0016937255859375 seconds
INFO:root:Cycle: 0, forward NEQ step: 31500, took: 0.002562999725341797 seconds
INFO:root:Cycle: 0, forward NEQ step: 32250, took: 0.001728057861328125 seconds
INFO:root:Cycle: 0, forward NEQ st

INFO:root:Cycle: 0, forward equil step: 58500, took: 0.0017118453979492188 seconds
INFO:root:Cycle: 0, forward equil step: 59250, took: 0.0016188621520996094 seconds
INFO:root:Cycle: 0, forward equil step: 60000, took: 0.0017819404602050781 seconds
INFO:root:Cycle: 0, forward equil step: 60750, took: 0.001821279525756836 seconds
INFO:root:Cycle: 0, forward equil step: 61500, took: 0.0017201900482177734 seconds
INFO:root:Cycle: 0, forward equil step: 62250, took: 0.001642465591430664 seconds
INFO:root:Cycle: 0, forward equil step: 63000, took: 0.0019197463989257812 seconds
INFO:root:Cycle: 0, forward equil step: 63750, took: 0.0017611980438232422 seconds
INFO:root:Cycle: 0, forward equil step: 64500, took: 0.0017430782318115234 seconds
INFO:root:Cycle: 0, forward equil step: 65250, took: 0.001752614974975586 seconds
INFO:root:Cycle: 0, forward equil step: 66000, took: 0.0011067390441894531 seconds
INFO:root:Cycle: 0, forward equil step: 66750, took: 0.0016672611236572266 seconds
INFO:ro

INFO:root:Cycle: 0, forward NEQ step: 8250, took: 0.002765655517578125 seconds
INFO:root:Cycle: 0, forward NEQ step: 9000, took: 0.002787351608276367 seconds
INFO:root:Cycle: 0, forward NEQ step: 9750, took: 0.002604961395263672 seconds
INFO:root:Cycle: 0, forward NEQ step: 10500, took: 0.0027315616607666016 seconds
INFO:root:Cycle: 0, forward NEQ step: 11250, took: 0.003032207489013672 seconds
INFO:root:Cycle: 0, forward NEQ step: 12000, took: 0.0026781558990478516 seconds
INFO:root:Cycle: 0, forward NEQ step: 12750, took: 0.002627134323120117 seconds
INFO:root:Cycle: 0, forward NEQ step: 13500, took: 0.0015592575073242188 seconds
INFO:root:Cycle: 0, forward NEQ step: 14250, took: 0.002154827117919922 seconds
INFO:root:Cycle: 0, forward NEQ step: 15000, took: 0.001580953598022461 seconds
INFO:root:Cycle: 0, forward NEQ step: 15750, took: 0.003004789352416992 seconds
INFO:root:Cycle: 0, forward NEQ step: 16500, took: 0.0029866695404052734 seconds
INFO:root:Cycle: 0, forward NEQ step: 1

...

INFO:root:Cycle: 9, forward equil step: 12000, took: 0.0017440319061279297 seconds
INFO:root:Cycle: 9, forward equil step: 12750, took: 0.0017547607421875 seconds
INFO:root:Cycle: 9, forward equil step: 13500, took: 0.001779794692993164 seconds
INFO:root:Cycle: 9, forward equil step: 14250, took: 0.0017576217651367188 seconds
INFO:root:Cycle: 9, forward equil step: 15000, took: 0.0015859603881835938 seconds
INFO:root:Cycle: 9, forward equil step: 15750, took: 0.0019655227661132812 seconds
INFO:root:Cycle: 9, forward equil step: 16500, took: 0.0017452239990234375 seconds
INFO:root:Cycle: 9, forward equil step: 17250, took: 0.0016973018646240234 seconds
INFO:root:Cycle: 9, forward equil step: 18000, took: 0.0017545223236083984 seconds
INFO:root:Cycle: 9, forward equil step: 18750, took: 0.0014812946319580078 seconds
INFO:root:Cycle: 9, forward equil step: 19500, took: 0.0016632080078125 seconds
INFO:root:Cycle: 9, forward equil step: 20250, took: 0.0016260147094726562 seconds
INFO:root:C

INFO:root:Cycle: 9, forward equil step: 87000, took: 0.001932382583618164 seconds
INFO:root:Cycle: 9, forward equil step: 87750, took: 0.0018529891967773438 seconds
INFO:root:Cycle: 9, forward equil step: 88500, took: 0.0016028881072998047 seconds
INFO:root:Cycle: 9, forward equil step: 89250, took: 0.0016846656799316406 seconds
INFO:root:Cycle: 9, forward equil step: 90000, took: 0.0015866756439208984 seconds
INFO:root:Cycle: 9, forward equil step: 90750, took: 0.001603841781616211 seconds
INFO:root:Cycle: 9, forward equil step: 91500, took: 0.0015592575073242188 seconds
INFO:root:Cycle: 9, forward equil step: 92250, took: 0.0017006397247314453 seconds
INFO:root:Cycle: 9, forward equil step: 93000, took: 0.001573801040649414 seconds
INFO:root:Cycle: 9, forward equil step: 93750, took: 0.001585245132446289 seconds
INFO:root:Cycle: 9, forward equil step: 94500, took: 0.0015926361083984375 seconds
INFO:root:Cycle: 9, forward equil step: 95250, took: 0.0017757415771484375 seconds
INFO:roo

INFO:root:Cycle: 9, forward NEQ step: 37500, took: 0.0025641918182373047 seconds
INFO:root:Cycle: 9, forward NEQ step: 38250, took: 0.003139972686767578 seconds
INFO:root:Cycle: 9, forward NEQ step: 39000, took: 0.002737283706665039 seconds
INFO:root:Cycle: 9, forward NEQ step: 39750, took: 0.0026907920837402344 seconds
INFO:root:Cycle: 9, Starting to equilibrate at lambda = 1
INFO:root:Cycle: 9, forward equil step: 0, took: 0.0034732818603515625 seconds
INFO:root:Cycle: 9, forward equil step: 750, took: 0.0016505718231201172 seconds
INFO:root:Cycle: 9, forward equil step: 1500, took: 0.0016338825225830078 seconds
INFO:root:Cycle: 9, forward equil step: 2250, took: 0.0013647079467773438 seconds
INFO:root:Cycle: 9, forward equil step: 3000, took: 0.0016145706176757812 seconds
INFO:root:Cycle: 9, forward equil step: 3750, took: 0.0016815662384033203 seconds
INFO:root:Cycle: 9, forward equil step: 4500, took: 0.0016524791717529297 seconds
INFO:root:Cycle: 9, forward equil step: 5250, took

INFO:root:Cycle: 9, forward equil step: 71250, took: 0.0010428428649902344 seconds
INFO:root:Cycle: 9, forward equil step: 72000, took: 0.0013284683227539062 seconds
INFO:root:Cycle: 9, forward equil step: 72750, took: 0.0010573863983154297 seconds
INFO:root:Cycle: 9, forward equil step: 73500, took: 0.0012805461883544922 seconds
INFO:root:Cycle: 9, forward equil step: 74250, took: 0.0010225772857666016 seconds
INFO:root:Cycle: 9, forward equil step: 75000, took: 0.001055002212524414 seconds
INFO:root:Cycle: 9, forward equil step: 75750, took: 0.001041412353515625 seconds
INFO:root:Cycle: 9, forward equil step: 76500, took: 0.0010766983032226562 seconds
INFO:root:Cycle: 9, forward equil step: 77250, took: 0.0010263919830322266 seconds
INFO:root:Cycle: 9, forward equil step: 78000, took: 0.0010418891906738281 seconds
INFO:root:Cycle: 9, forward equil step: 78750, took: 0.0011150836944580078 seconds
INFO:root:Cycle: 9, forward equil step: 79500, took: 0.0010395050048828125 seconds
INFO:r

INFO:root:Cycle: 9, forward NEQ step: 21000, took: 0.0026397705078125 seconds
INFO:root:Cycle: 9, forward NEQ step: 21750, took: 0.003254413604736328 seconds
INFO:root:Cycle: 9, forward NEQ step: 22500, took: 0.0017600059509277344 seconds
INFO:root:Cycle: 9, forward NEQ step: 23250, took: 0.0026657581329345703 seconds
INFO:root:Cycle: 9, forward NEQ step: 24000, took: 0.002725362777709961 seconds
INFO:root:Cycle: 9, forward NEQ step: 24750, took: 0.002728700637817383 seconds
INFO:root:Cycle: 9, forward NEQ step: 25500, took: 0.0026636123657226562 seconds
INFO:root:Cycle: 9, forward NEQ step: 26250, took: 0.0028939247131347656 seconds
INFO:root:Cycle: 9, forward NEQ step: 27000, took: 0.003150463104248047 seconds
INFO:root:Cycle: 9, forward NEQ step: 27750, took: 0.0026378631591796875 seconds
INFO:root:Cycle: 9, forward NEQ step: 28500, took: 0.0030219554901123047 seconds
INFO:root:Cycle: 9, forward NEQ step: 29250, took: 0.002687215805053711 seconds
INFO:root:Cycle: 9, forward NEQ step

In [26]:
pos

Quantity(value=array([[-1.24167823,  0.30676174, -0.46729298],
       [-1.25169248,  0.24277972, -0.37961732],
       [-1.29340592,  0.30188284, -0.2980832 ],
       [-1.15795054,  0.19497466, -0.35118976],
       [-1.33051117,  0.1192096 , -0.41319636],
       [-1.31645312,  0.00903349, -0.37425861],
       [-1.43267547,  0.1516972 , -0.49872383],
       [-1.44136362,  0.24235004, -0.54239968],
       [-1.53628385,  0.05606706, -0.5434297 ],
       [-1.49265562, -0.041554  , -0.52227066],
       [        nan,         nan,         nan],
       [        nan,         nan,         nan],
       [        nan,         nan,         nan],
       [-1.59938474,  0.04882936, -0.67175988],
       [-1.5762857 ,  0.15211093, -0.75158854],
       [        nan,         nan,         nan],
       [-1.66743348, -0.06000669, -0.71275894],
       [-1.66898432, -0.12458407, -0.63511642],
       [-1.70701727, -0.09695206, -0.84503007],
       [-1.7349208 , -0.2022882 , -0.84244169],
       [-1.80202839, -0.0